<a href="https://colab.research.google.com/github/yuce-8/alpha/blob/main/YUCE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YUCE-8.tech API documentation
**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install requests

import json
import requests

## Connect to YUCE-8
this is subject to change in the next versions

In [2]:
class GCF():
  def __init__(self):
    #self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_api'
    self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api'
    print('YUCE-8 ready @ ', self.url)

  def push(self, json_obj, return_result=False):
    r = requests.post(self.url, json=json_obj)
    if return_result:
      return r

yuce_8 = GCF()

YUCE-8 ready @  https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api


## Test connection
the test will return the current backend version

In [3]:
def get_current_backend_version():
   req = yuce_8.push({'action': 'get_version'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']


print(get_current_backend_version())

v0.1.3 alpha


## Create a test key
Key can be reused for 50 calls

In [4]:
def create_test_key(your_mail_address):
   req = yuce_8.push({'action': 'create_test_license', 'user_name': your_mail_address}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']



key = create_test_key('torsten@yuce-8.info')
print('our new key = ', key)

our new key =  a0a669b3-fe56-4014-bcc1-25c4ed3a51d0


## List all available forecasts

In [5]:
def get_available_forecasts():
   req = yuce_8.push({'action': 'get_available_forecasts'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']

get_available_forecasts()

[{'BTC-USD': ['1d'], '^NSEI': ['1d']}]

## Get latest forecast

In [6]:
def get_latest_forecast(key, symbol, interval):
   req = yuce_8.push(
      {
        'action': 'get_forecast',
        'key': key,
        'symbol': symbol,
        'interval': interval
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']['f_0']['yuce_8'], req.json()['msg']['api_calls_left_with_this_key'] 
   else:
     return ';-( ' + req.json()['msg'], -1

# get the forecast:
f_0, calls_left = get_latest_forecast(key, 'BTC-USD', '1d')

# print out the content:
if calls_left != -1:
  print('how many API calls are left?', calls_left)
  print('when was the forecast created?', f_0['forecast_created_at'])
  print('what is the last quote that was used to forecast?', f_0['last_quote'], ' @ ', f_0['last_date'])
  print('what is the name of the magic model that was used to forecast?', f_0['model_name'], 'last trained @', f_0['training_date'])
  print('what is the forecasted top range?', f_0['quotes_forecast_top'])
  print('what is the forecasted bottom range?', f_0['quotes_forecast_bottom'])

  # display the forecast timestep per timestep:
  print('timestep', '0 = ', f_0['last_quote'], ' @ ', f_0['last_date'])
  quote_0 = f_0['last_quote']
  top_range = f_0['quotes_forecast_top']
  bottom_range = f_0['quotes_forecast_bottom']

  for i in range(0, len(top_range)):
    forecast_top_at_i = top_range[i]
    forecast_bottom_at_i = bottom_range[i] 
    
    # see if it is a bearish or bullish forecast
    bearish = forecast_top_at_i <= quote_0

    print('timestep', i+1, 'should be w/in', ']' if bearish else '[', forecast_bottom_at_i, ', ', forecast_top_at_i, ']' if bearish else '[')

else:
  print(f_0)

how many API calls are left? 49
when was the forecast created? 2021-02-16 03:02:33.676614+01:00
what is the last quote that was used to forecast? 47909.328125  @  2021-02-11 00:00:00
what is the name of the magic model that was used to forecast? model typ Y-3 last trained @ 2021-02-14 05:32:05
what is the forecasted top range? [48244.693421874996, 48584.8496515625, 48964.24164156246, 48964.24164156246, 48964.24164156246, 49460.748695759525, 49460.748695759525, 50200.70111238333, 50200.70111238333, 50200.70111238333, 50200.70111238333, 50200.70111238333, 50200.70111238333]
what is the forecasted bottom range? [47909.328125, 48244.693421874996, 48584.8496515625, 48584.8496515625, 48584.8496515625, 48964.24164156246, 48964.24164156246, 49460.748695759525, 49460.748695759525, 49460.748695759525, 49460.748695759525, 49460.748695759525, 49460.748695759525]
timestep 0 =  47909.328125  @  2021-02-11 00:00:00
timestep 1 should be w/in [ 47909.328125 ,  48244.693421874996 [
timestep 2 should be 

## Get backtesting statistics

In [7]:
def get_backtest_statistics(key):
   req = yuce_8.push(
      {
        'action': 'get_backtest_statistics',
        'key': key
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg'], req.json()['msg']['api_calls_left_with_this_key'] 
   else:
     return ';-( ' + req.json()['msg'], -1

# pull the backtesting results:
results, calls_left = get_backtest_statistics(key)
print(calls_left)
print(results)

48
{'api_calls_left_with_this_key': 48, 'key': 'a0a669b3-fe56-4014-bcc1-25c4ed3a51d0', 'results': [{'confidence intervals 80%': [[1.014734290106172, 1.0263995905090624], [1.0238306195058655, 1.0385423033914734], [1.0321786411122889, 1.051000117508927], [1.0452015914114763, 1.0670036755410384], [1.0606982249386607, 1.0861075383059682], [1.071097395680754, 1.0990137836710527], [1.083293261973727, 1.1153499455763218], [1.0949766321334347, 1.1303003148880497], [1.1090567240883633, 1.1472320427725986], [1.1192049454349562, 1.1577523280483202], [1.1257500213492222, 1.1663229900490932], [1.1395606080639615, 1.1836062391650424], [1.1486298398615309, 1.1950922651647142]], 'confidence intervals 95%': [[1.0116190978809696, 1.0295147827342648], [1.0199018974260872, 1.0424710254712517], [1.0271524081896366, 1.0560263504315792], [1.0393793939002325, 1.0728258730522822], [1.053912724975886, 1.092893038268743], [1.0636423870809266, 1.10646879227088], [1.0747325967464658, 1.123910610803583], [1.0855435

# DISCLAIMER

**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**


The information on this site is provided by YUCE-8 and the author and it is not to be construed as an offer or solicitation for the purchase or sale of any financial instrument or the provision of an offer to provide investment services. Information, opinions and comments contained on this site are not under the scope of investment advisory services. Investment advisory services are given according to the investment advisory contract, signed between the intermediary institutions, portfolio management companies, investment banks and the clients. Opinions and comments contained in this site reflect the personal views of the analysts / AI algorithms who supplied them. The investments discussed or recommended in this report may involve significant risk, may be illiquid and may not be suitable for all investors. Therefore, making decisions with respect to the information in this report may cause inappropriate results and a total loss.
Futures and forex trading contains substantial risk and is not for every investor. An investor could potentially lose all or more than the initial investment. Risk capital is money that can be lost without jeopardizing ones’ financial security or life style. Only risk capital should be used for trading and only those with sufficient risk capital should consider trading. Past performance is not necessarily indicative of future results
Hypothetical performance results have many inherent limitations, some of which are described below. no representation is being made that any account will or is likely to achieve profits or losses similar to those shown; in fact, there are frequently sharp differences between hypothetical performance results and the actual results subsequently achieved by any particular trading program. One of the limitations of hypothetical performance results is that they are generally prepared with the benefit of hindsight. In addition, hypothetical trading does not involve financial risk, and no hypothetical trading record can completely account for the impact of financial risk of actual trading. for example, the ability to withstand losses or to adhere to a particular trading program in spite of trading losses are material points which can also adversely affect actual trading results. There are numerous other factors related to the markets in general or to the implementation of any specific trading program which cannot be fully accounted for in the preparation of hypothetical performance results and all which can adversely affect trading results.
Testimonials appearing on this website may not be representative of other clients or customers and is not a guarantee of future performance or success.

FOR ALL TRADERS, WE BOLDLY IMPLORE THE FOLLOWING:

Always do your own Research
THIS CONTENT IS INTENDED TO BE USED FOR EDUCATIONAL PURPOSES ONLY.  WE ENCOURAGE ALL TRADERS TO DO THEIR OWN ANALYSIS BEFORE MAKING ANY INVESTMENT CHOICES.
 

Not Investment Advice
YUCE-8 IS A DATA ANALYTICS PORTAL. YUCE-8 IS NOT A BROKER/DEALER, WE ARE NOT AN INVESTMENT ADVISOR, WE HAVE NO ACCESS TO NON-PUBLIC INFORMATION ABOUT PUBLICLY TRADED COMPANIES, AND THIS IS NOT A PLACE FOR THE GIVING OR RECEIVING OF FINANCIAL ADVICE, ADVICE CONCERNING INVESTMENT DECISIONS OR TAX OR LEGAL ADVICE. WE ARE NOT REGULATED BY THE SEC OR ANY FINANCIAL SERVICES AUTHORITY.

 
No Reliance
ACCORDINGLY, WE WILL NOT BE LIABLE, WHETHER IN CONTRACT, TORT (INCLUDING NEGLIGENCE) OR OTHERWISE, IN RESPECT OF ANY DAMAGE, EXPENSE OR OTHER LOSS YOU MAY SUFFER ARISING OUT OF SUCH INFORMATION OR ANY RELIANCE YOU MAY PLACE UPON SUCH INFORMATION. ANY ARRANGEMENTS BETWEEN YOU AND ANY THIRD PARTY CONTACTED VIA THE WEBSITE ARE AT YOUR SOLE RISK.



# Terms and Conditions

ACCESS TO YUCE-8 AND ANY INFORMATION ON THE SITE IS NOT DIRECTED AT ANY PERSON IN ANY JURISDICTION WHERE (BY REASON OF THAT PERSON’S NATIONALITY, RESIDENCE OR OTHERWISE) IT IS PROHIBITED TO ACCESS THE SITE OR DIRECT SUCH INFORMATION FOR ANY REASON. IF YOU ARE A PERSON IN A JURISDICTION IN WHICH SUCH PROHIBITIONS APPLY, YOU MUST NOT ACCESS THIS SITE.

Disclaimer of Warranties
YUCE-8 & THE SITE IS PROVIDED “AS IS” AND YOUR ACCESS AND USE OF IT IS AT YOUR OWN RISK. TO THE EXTENT PERMITTED BY LAW AND UNLESS OTHERWISE SPECIFICALLY SET OUT IN THIS AGREEMENT, WE MAKE NO WARRANTIES, REPRESENTATIONS OR UNDERTAKINGS IN RESPECT THEREOF AND ALL WARRANTIES REPRESENTATIONS, UNDERTAKINGS, OR OTHER TERMS OF ANY KIND, WHETHER EXPRESS OR IMPLIED BY STATUTE, CUSTOM OR USAGE (INCLUDING WITHOUT LIMITATION, WARRANTIES AS TO SATISFACTORY QUALITY, FITNESS FOR PURPOSE OR SKILL AND CARE) RELATING TO THE SITE AND THE INFORMATION CONTAINED IN OR OMITTED FROM THE SITE ARE EXCLUDED.

WE MAKE NO WARRANTY THAT THE SITE WILL BE ERROR FREE OR UNINTERRUPTED, THAT THE INFORMATION OBTAINED FROM THE SITE WILL BE ACCURATE, COMPLETE, CURRENT, OR RELIABLE, THAT THE QUALITY OF THE SITE WILL BE SATISFACTORY TO YOU, OR THAT ERRORS OR DEFECTS WILL BE CORRECTED

Limitation of Liability
TO THE FULLEST EXTENT PERMITTED BY LAW, YUCE-8 (AND ITS AFFILIATES, AGENTS OR LICENSORS) EXCLUDES ALL LIABILITY, WHETHER IN CONTRACT, TORT, NEGLIGENCE, MISREPRESENTATION OR OTHERWISE, FOR ANY DIRECT OR INDIRECT LOSS OR DAMAGE CAUSED, DIRECTLY OR INDIRECTLY, BY YOUR USE OF THE SITE OR THE INFORMATION CONTAINED ON THE SITE OR PROVIDED TO YOU PURSUANT TO YOUR USE OF THE SITE.

IN NO CIRCUMSTANCE SHALL YUCE-8 (OR ANY OF ITS AFFILIATES, AGENTS OR LICENSORS) BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL OR CONSEQUENTIAL LOSS OR DAMAGES (INCLUDING, WITHOUT LIMITATION, LOSS OF PROFIT, BUSINESS OPPORTUNITY, LOSS OF GOODWILL, ANTICIPATED SAVINGS OR DATA) ARISING FROM: (I) YOUR INABILITY TO USE THE SITE AT ANY TIME OR ANY ERROR IN THE PROVISION OF THE SITE; (II) UNAUTHORIZED ACCESS TO OR ALTERATION OF THE SITE OR ANY INFORMATION OR DATA TRANSMITTED IN RELATION TO THE SITE; (III) ANY ERROR, FAULT, COMPUTER VIRUS OR OTHER COMPUTER OR DATA-CORRUPTING OR DATA-DAMAGING MATERIAL TRANSMITTED THROUGH THE SITE; (IV) THIRD PARTY SERVICES OUTSIDE YUCE-8’S REASONABLE CONTROL (INCLUDING, BUT NOT LIMITED TO, TELEPHONE AND BROWSER SERVICES AND YOUR USE OF EMAIL COMMUNICATIONS TO CONTACT OR INSTRUCT US, AND ANY DELAY IN OUR RECEIPT OR CONFIRMATION OF SUCH A COMMUNICATION, ANY INTERCEPTION OF OR CHANGES TO SUCH A COMMUNICATION AND ANY REASONABLE DELAY IN ACTING ON SUCH A COMMUNICATION), WHETHER SUCH SERVICES ARE USED BY YOU TO ACCESS THE SITE OR OTHERWISE; (V) YOUR USE OF AND/OR RELIANCE ON OTHER WEB SITES TO WHICH YOU HAVE GAINED ACCESS BY MEANS OF HYPERLINKS PUBLISHED ON THE SITE; OR (VI) ANY INACCURATE INFORMATION, MATERIAL OR OPINIONS PUBLISHED ON THE SITE (OR ACCESSED VIA THE SITE) OR YOUR RELIANCE THEREON. FOR THE AVOIDANCE OF DOUBT THIS PROVISION SHALL APPLY WHETHER OR NOT THE RELEVANT LOSS OR DAMAGE ARISES BY REASON OF YUCE-8’S NEGLIGENCE OR IN CONTRACT, TORT, MISREPRESENTATION OR OTHERWISE.

YUCE-8’S LIABILITY IS LIMITED EVEN IF IT HAS BEEN ADVISED OF THE POSSIBILITY OF THE DAMAGES THAT YOU SUFFER OR IF ANY REMEDY YOU HAVE FAILS ITS ESSENTIAL PURPOSE. THIS DISCLAIMER OF LIABILITY APPLIES TO ANY AND ALL DAMAGES OR INJURY, INCLUDING THOSE CAUSED BY ANY FAILURE OF PERFORMANCE, ERROR, OMISSION, INTERRUPTION, DELETION, DEFECT, DELAY IN OPERATION OR TRANSMISSION, COMPUTER VIRUS, COMMUNICATION LINE FAILURE, THEFT, DESTRUCTION OR UNAUTHORIZED ACCESS TO, ALTERATION OF OR USE OF ANY ASSET, WHETHER FOR BREACH OF CONTRACT, TORTIOUS BEHAVIOR, NEGLIGENCE OR UNDER ANY OTHER CAUSE OF ACTION. IF YOU HAVE A DISPUTE WITH US OR ARE DISSATISFIED WITH THE SITE, TERMINATION OF YOUR USE OF THE SITE IS YOUR SOLE REMEDY. WE HAVE NO OTHER OBLIGATION, LIABILITY, OR RESPONSIBILITY TO YOU.

Use Restrictions
AS A VISITOR TO THE SITE, YOU MAY VIEW THE SITE AND YOU ARE PERMITTED TO PRINT HARD COPIES OF MATERIAL ON IT FOR YOUR PERSONAL, NON-COMMERCIAL USE. ALL OTHER COPYING OR DISTRIBUTION OF ANY PART OF THE SITE WHETHER IN ELECTRONIC, HARD COPY OR OTHER FORMAT IS PROHIBITED. ANY COPIES MUST NOT ALTER THE ORIGINAL CONTENT AND MUST REPRODUCE OUR COPYRIGHT NOTICES.

YOUR USE OF THE SITE IS CONDITIONED ON YOUR FULL COMPLIANCE WITH THIS AGREEMENT INCLUDING, WITHOUT LIMITATION, THE RULES OF CONDUCT SET FORTH IN THIS SECTION AND YOUR FAILURE TO COMPLY WITH SUCH RULES MAY RESULT IN TERMINATION OF YOUR ACCESS TO THE SITE PURSUANT TO THIS AGREEMENT. YOU AGREE THAT WHEN USING THE SITE YOU WILL NOT:

TRANSMIT BY ANY MEANS ANY SOFTWARE, VIRUS, MALWARE, PROGRAM, CODE, FILE, OR OTHER MATERIAL INTENDED TO INTERRUPT, DISRUPT, ALTER, DESTROY, OR LIMIT ANY PART OF THE SITE;

USE ANY ROBOT, SPIDER, SCRIPT, OR ANY MANUAL OR AUTOMATED APPLICATION OR MEANS TO EXTRACT, DOWNLOAD, RETRIEVE, INDEX, MINE, SCRAPE, REPRODUCE, OR CIRCUMVENT THE PRESENTATION, OPERATION, OR INTENDED USE OF ANY FEATURE, FUNCTION, OR PART OF THE SITE;

FRAME OR MIRROR ANY PART OF THE SITE WITHOUT OUR EXPRESS PRIOR WRITTEN CONSENT;

MODIFY, ADAPT, TRANSLATE, REVERSE ENGINEER, DECOMPILE OR DISASSEMBLE ANY PORTION OF THE SITE;

POST OR SUBMIT ANY OFFENSIVE, DEFAMATORY, THREATENING, INDECENT, HATEFUL, RACIST, SEXIST, PORNOGRAPHIC, OR OTHERWISE INAPPROPRIATE OR OBSCENE CONTENT (AS DETERMINED BY US IN OUR SOLE DISCRETION);

COPY, DOWNLOAD, DISTRIBUTE, TRANSMIT, UPLOAD, OR TRANSFER CONTENT FROM THE SITE WITHOUT OUR PRIOR WRITTEN PERMISSION OR AS OTHERWISE PERMITTED UNDER THIS AGREEMENT;

RESELL, SUB-LICENSE, OR LEASE ANY OF THE CONTENT ON THE SITE;

USE THE SITE TO MAKE UNSOLICITED OFFERS, REQUESTS, ADVERTISEMENTS, OR SPAM;

IMPERSONATE OR PRETEND TO BE ANYONE ELSE BUT YOU; FALSELY STATE OR OTHERWISE MISREPRESENT YOUR AFFILIATION WITH ANY PERSON OR ENTITY IN CONNECTION WITH THE SITE; OR EXPRESS OR IMPLY THAT WE ENDORSE ANY STATEMENT YOU MAKE;

REMOVE ANY COPYRIGHT, TRADEMARK OR OTHER PROPRIETARY NOTICE OR LEGEND CONTAINED ON THE SITE;

VIOLATE OR INFRINGE UPON THE PRIVACY, PUBLICITY, INTELLECTUAL PROPERTY, OR OTHER PROPRIETARY RIGHTS OF THIRD PARTIES; OR

ENGAGE IN ANY ACTIVITY THAT IS CRIMINAL OR TORTIOUS IN NATURE, OR OTHERWISE VIOLATES THE LAW OR RIGHTS OF ANOTHER INCLUDING, WITHOUT LIMITATION, HACKING, PHISHING, FRAUD, DEFAMING, ABUSING, HARASSING, OR THREATENING.

IF WE TERMINATE OR SUSPEND YOUR ACCOUNT FOR ANY REASON, YOU ARE PROHIBITED FROM REGISTERING AND CREATING A NEW ACCOUNT UNDER YOUR NAME, A FAKE OR BORROWED NAME, OR THE NAME OF ANY THIRD PARTY, EVEN IF YOU MAY BE ACTING ON BEHALF OF THE THIRD PARTY. IN ADDITION TO TERMINATING OR SUSPENDING YOUR ACCOUNT, WE RESERVE THE RIGHT TO TAKE APPROPRIATE LEGAL ACTION, INCLUDING WITHOUT LIMITATION PURSUING CIVIL, CRIMINAL, AND INJUNCTIVE REDRESS.


Changes
YUCE-8 RESERVES THE RIGHT TO MAKE ANY CHANGES TO THESE TERMS AND CONDITIONS AT ANY TIME AND IT IS YOUR RESPONSIBILITY TO CHECK FOR ANY CHANGES TO THESE TERMS AND CONDITIONS. IF WE CHANGE THIS AGREEMENT, WE WILL UPDATE THE EFFECTIVE DATE LISTED BELOW. YOUR CONTINUED USE OF THE SITE MEANS THAT YOU AGREE WITH THE UPDATED AGREEMENT. IF YOU DO NOT AGREE WITH OUR UPDATED AGREEMENT, YOU MAY NOT USE OUR SITE.

Miscellaneous
IF A COMPETENT COURT OF LAW OR AUTHORITY FINDS THAT ANY PART OF THESE TERMS AND CONDITIONS IS INVALID, UNLAWFUL OR UNENFORCEABLE TO ANY EXTENT, THE REMAINDER OF THESE TERMS AND CONDITIONS SHALL CONTINUE TO BE VALID AND ENFORCEABLE TO THE FULLEST EXTENT PERMITTED BY UNITED STATES LAW.

THESE TERMS AND CONDITIONS ARE GOVERNED BY THE LAWS OF THE STATE OF NEW YORK AND UNITED STATES LAW. THE COURTS OF NEW YORK SITUATED IN MANHATTAN SHALL HAVE EXCLUSIVE JURISDICTION IN RELATION TO ANY DISPUTE ARISING OUT OF YOUR USE OF THE SITE OR ARISING FROM OR CONNECTED WITH THESE TERMS AND CONDITIONS AND YOU HEREBY SUBMIT TO THE EXCLUSIVE JURISDICTION OF THE COURTS OF NEW YORK. YOU AGREE THAT YOU WILL NOT FILE A CLASS ACTION, OR PARTICIPATE IN A CLASS ACTION AGAINST US.

Investment Solicitation
THE INFORMATION AND MATERIALS CONTAINED IN THIS WEBSITE ARE PRESENTED FOR INFORMATIONAL PURPOSES ONLY, AND DO NOT CONSTITUTE AN OFFER TO SELL OR A SOLICITATION OF AN OFFER TO BUY INTERESTS IN ANY FUND OR OTHER INVESTMENT. NONE OF SUCH INFORMATION AND MATERIALS SHOULD BE CONSTRUED AS PROVIDING ANY TYPE OF INVESTMENT OR OTHER ADVICE TO YOU, NOR SHOULD YOU CONSIDER IT AS A SOLICITATION, RECOMMENDATION, ENDORSEMENT OR OFFER TO YOU TO BUY OR SELL ANY SECURITY OR OTHER FINANCIAL INSTRUMENT, EFFECT ANY TRANSACTION OR CONCLUDE ANY LEGAL ACT OF ANY KIND WHATSOEVER.

YUCE-8 IS NOT REGISTERED WITH THE US SECURITIES AND EXCHANGE COMMISSION (“SEC”) OR ANY STATE SECURITIES REGULATORY AUTHORITY AS A SECURITIES BROKER, DEALER OR INVESTMENT ADVISOR. WE ARE NOT LICENSED TO GIVE ANY KIND OF INVESTMENT ADVICE. WE DO NOT GIVE INVESTMENT ADVICE IN ANY MANNER. WE DO NOT PROVIDE LEGAL, TAX, ACCOUNTING, OR INVESTMENT ADVICE. SINCE YOUR SITUATION IS UNIQUE, YOU SHOULD ALWAYS CONSULT WITH A QUALIFIED PROFESSIONAL MAKING FINANCIAL DECISIONS.